In [1]:
import pandas as pd
import random, time
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import franchiseplayers
from nba_api.live.nba.endpoints import scoreboard
from datetime import datetime

In [2]:
# player1 = input('enter player 1: ')
player1= 'Russell Westbrook'
player2 = "Zion Williamson"

In [3]:
# TODO figure out a better solution for same name players (the gerald henderson problem)
p1Dict = players.find_players_by_full_name(player1)[-1]
# time.sleep(.600)
p2Dict = players.find_players_by_full_name(player2)[-1]
# time.sleep(.600)

p1Career = playercareerstats.PlayerCareerStats(player_id=p1Dict['id']).get_data_frames()
# time.sleep(.600)
p2Career = playercareerstats.PlayerCareerStats(player_id=p2Dict['id']).get_data_frames()
# time.sleep(.600)

In [4]:
p1_teammates = pd.DataFrame(columns=['PLAYER', 'TeamID'])
for season in range(p1Career[0].shape[0]):
    temp_teammates = commonteamroster.CommonTeamRoster(team_id=p1Career[0]['TEAM_ID'][season],
                                                        season=p1Career[0]['SEASON_ID'][season]).get_data_frames()[0]
    time.sleep(.600)
    p1_teammates = pd.concat([p1_teammates, temp_teammates])
p1_teammates.PLAYER.nunique()

150

In [5]:
p2_teammates = pd.DataFrame(columns=['PLAYER', 'TeamID'])
for season in range(p2Career[0].shape[0]):
    temp_teammates = commonteamroster.CommonTeamRoster(team_id=p2Career[0]['TEAM_ID'][season],
                                                        season=p2Career[0]['SEASON_ID'][season]).get_data_frames()[0]
    time.sleep(.600)
    p2_teammates = pd.concat([p2_teammates, temp_teammates])
p2_teammates.PLAYER.nunique()

44

In [6]:
p1_teammates_u = p1_teammates.groupby(['PLAYER'], as_index=False).first()[['PLAYER',]] #'TeamID']]
p2_teammates_u = p2_teammates.groupby(['PLAYER'], as_index=False).first()[['PLAYER',]] #'TeamID']]

In [7]:
common_teammates = pd.merge(p1_teammates_u, p2_teammates_u, how='inner', on='PLAYER')
print(f"{player1} and {player2} have {common_teammates.shape[0]} teammates in common.")
print(common_teammates['PLAYER'].tolist())

Russell Westbrook and Zion Williamson have 2 teammates in common.
['Steven Adams', 'Wenyen Gabriel']
